
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Ce TD a été développé avec l'aide précieuse de :
- Sanae Lofti, candidate à la maîtrise,
- Amine Bellahsen, candidat à la maîtrise.<br/>

Tous les deux étaient inscrits au cours à l'automne 2018.

---


# TD 8 : Filtre anti-spam 
___

### Description

Dans ce travail dirigé, vous aurez l'occasion de développer un filtre anti-spam basé sur la classification bayésienne naïve. Que la présence ou l'absence de certains mots sera considérée comme variable explicative.

### Données

Les données exploitées dans ce TD correspondents à des messages électroniques authentiques d'un employé de la compagnie Enron. Vous pouvez télécharger le jeux de données à partir du site web du cours. Le fichier doit être décompressé dans un dossier nommé *data* du répertoire courant de votre calepin Jupyter. 

Notez que les messages électroniques de 158 employés de la compagnie Enron ont été récupérés par la Federal Energy Regulatory Commission pendant la commission d'enquête qui a eu lieu après l'effondrement de la compagnie. Dans ce TD, nous n'utilisons que les messages d'un seul employé. Vous pouvez récupérer le jeu de données entier à l'adresse suivante https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz.



### Sommaire:
___


[Préliminaires](#unit0)  
[Exercice 1](#unit1)  
[Exercice 2](#unit2)  
[Exercice 3](#unit3)  


In [1]:
using StatsBase, Statistics, DataFrames

# Préliminaires  <a id = "unit0" > </a > 


Les codes de cette section permettent de traiter les fichiers textes correspondant aux messages électroniques avant de pouvoir répondre aux questions.

### Construction des ensembles d'entraînement et de test

Les messages électroniques de l'utilisateur nommé *Enron1* sont répartis dans un dossier *ham* et un dossier *spam*. La cellule de code permet d'extraire aléatoirement des courriels et des pourriels pour l'ensemble d'entraînement. On prend le $2/3$ des messages pour constituer l'échantillon d'entraînement.

Le dictionnaire ``TrainSet`` contient deux champs :
- *:Ham* qui contient la liste des fichiers textes correspondants aux courriels de l'ensemble d'entraînement. 
- *:Spam* qui contient la liste des fichiers textes correspondants aux pourriels de l'ensemble d'entraînement.

Le dictionnaire ``TestSet`` contient les mêmes deux champs mais pour l'ensemble de test.

In [2]:
# Récupération des noms de fichier de tous les hams
filesdir = "data/enron1/ham/"
filename_ham = filesdir.*readdir(filesdir)

# Récupération des noms de fichier de tous les spams
filesdir = "data/enron1/spam/"
filename_spam = filesdir.*readdir(filesdir)

# nombre de fichiers pour contruire l'ensemble d'entraînement des hams
n = Int(round(2/3*length(filename_ham)))

# sélection aléatoire des fichiers de l'ensemble d'entraînement
ham_train = Array{String}(undef,n)
sample!(filename_ham, ham_train, replace=false, ordered=false)
ham_test = setdiff(filename_ham, ham_train)


# nombre de fichiers pour contruire l'ensemble d'entraînement des spams
n = Int(round(2/3*length(filename_spam)))

# sélection aléatoire des fichiers de l'ensemble d'entraînement
spam_train = Array{String}(undef,n)
sample!(filename_spam, spam_train, replace=false, ordered=false)
spam_test = setdiff(filename_spam, spam_train)


# Sauvegarder les ensembles d'entraînement et de test dans des dictionnaires
TrainSet = Dict( :Ham => ham_train, :Spam => spam_train)
TestSet = Dict( :Ham => ham_test, :Spam => spam_test)


Dict{Symbol,Array{String,1}} with 2 entries:
  :Ham  => ["data/enron1/ham/0003.1999-12-14.farmer.ham.txt", "data/enron1/ham/…
  :Spam => ["data/enron1/spam/0017.2003-12-18.GP.spam.txt", "data/enron1/spam/0…

## Dénombrement des mots contenus dans les courriels

Cette cellule permet de dénombrer le nombre de courriels où chaque mot est présent. Le dictionnaire ``ham_wordcounts`` indique le nombre de courriels dans lesquels cahque mot apparaît.

La fonction *wordlisting* prend en entrée le chemin d'accès d'un fichier texte et sort la liste de mots présents du fichier. Si un mot apparaît plus d'une fois dans le fichier, la fonction ne sort que la présence de ce mot, pas la quantité de fois où il apparaît.

La fonction *wordcounting* prend en entrée une matrice de liste de mots. La fonction dénombre le nombre de lignes où le mot apparaît.

In [3]:
"""
wordlisting : Cette fonction transforme un fichier texte en une liste de mots. Le nombre de fois que
              les mots apparaîssent ne sont pas répertorié.
Input: filename::String le chemin du fichier texte 
Output: Array{Sting} Liste des mots contenus dans le fichier texte
"""

function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
wordcounting : À partir d'un Array of Array contenant la liste des mots d'un ensemble de fichiers texte, la fonction 
               retourne le nombre de fois où chaque mot est présent.
Input: Un array correspondant à la liste des mots pour chacun des fichiers texte 
Output: Dictionnaire compilant le nombre de ligne dans lesquelles chacun des mots apparaît.
"""

function wordcounting(A::Array{Array{SubString{String},1},1})
   
    words = vcat(A...)

    wordcounts = Dict{String,Int64}()

    for word in words
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end

wordcounting (generic function with 1 method)

In [4]:
ham_wordlist = wordlisting.(TrainSet[:Ham])
ham_wordcounts = wordcounting(ham_wordlist)

Dict{String,Int64} with 13861 entries:
  "improvemnts" => 1
  "henry"       => 4
  "cuss"        => 1
  "hampshire"   => 3
  "tnpc"        => 1
  "rhonda"      => 9
  "msnbc"       => 1
  "gathered"    => 7
  "bretta"      => 1
  "acton"       => 65
  "underground" => 1
  "budgeted"    => 1
  "november"    => 103
  "backup"      => 6
  "stress"      => 4
  "caught"      => 1
  "undernom"    => 1
  "rectified"   => 1
  "premature"   => 1
  "fountain"    => 1
  "hicks"       => 1
  "package"     => 22
  "qnec"        => 1
  "gay"         => 18
  "replacement" => 6
  ⋮             => ⋮

## Dénombrement des mots contenus dans les pourriels

Cette cellule permet de dénombrer le nombre de courriels où chaque mot est présent. Le dictionnaire ``spam_wordcounts`` indique le nombre de courriels dans lesquels cahque mot apparaît.

Les fonctions *wordlisting* et *wordcounting* sont utilisées.

In [5]:
spam_wordlist = wordlisting.(TrainSet[:Spam])
spam_wordcounts = wordcounting(spam_wordlist)

Dict{String,Int64} with 28851 entries:
  "upsurge"        => 1
  "vvmiaml"        => 1
  "null"           => 2
  "grp"            => 2
  "ucewn"          => 1
  "inattentive"    => 1
  "duma"           => 1
  "bumbum"         => 1
  "henry"          => 1
  "cvfwyks"        => 1
  "mycology"       => 1
  "partl"          => 2
  "aslpp"          => 1
  "anaerobic"      => 1
  "youlaagan"      => 1
  "jemimajones"    => 1
  "corrodible"     => 1
  "hampshire"      => 1
  "brandt"         => 2
  "maggotadvances" => 1
  "whiz"           => 1
  "vrmtk"          => 1
  "il"             => 9
  "neumann"        => 1
  "soapstone"      => 1
  ⋮                => ⋮

# Exercice 1 <a id = "unit1" > </a > 

Considérez le mot ***http*** comme variable explicative pour classer les messages électroniques en courriels et pourriels.

### a) Si un message contient le mot ***http***, quelle est la probabilité que ce message soit un pourriel ?

In [6]:
# nombre de courriels dans l'ensemble d'entraînement
n₀ = length(TrainSet[:Ham])

# nombre de pourriels dans l'ensemble d'entraînement
n₁ = length(TrainSet[:Spam])

# nombre de messages de l'ensemble d'entraînement
n = n₀ + n₁

# nombre de courriels où le mot http apparaît
n₀₁ = ham_wordcounts["http"]

# nombre de pourriels où le mot http apparaît
n₁₁ = spam_wordcounts["http"]

q_spam = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2)

# constante de normalisation
c = q_spam + q_ham

p_spam = q_spam/c

println("Si le mot http est présent dans le message, il y a une prob de $p_spam que ce soit un spam.")

Si le mot http est présent dans le message, il y a une prob de 0.7999055423576151 que ce soit un spam.


### b) Si un message ne contient pas le mot ***http***, quelle est la probabilité que ce message soit un pourriel ?

In [7]:
q_spam = (n₁+1)/(n+2) * (n₁-n₁₁+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀-n₀₁+1)/(n₀+2)

# constante de normalisation
c = q_spam + q_ham

p_spam = q_spam/c

println("Si le mot http est présent dans le message, il y a une prob de $p_spam que ce soit un spam.")

Si le mot http est présent dans le message, il y a une prob de 0.2205109731260497 que ce soit un spam.


### c) Filtrez tous les courriels de l'échantillon de test. Quelle est la proportion de courriels classés comme pourriels?

In [8]:
fauxPositifs = Bool[]

for filename in TestSet[:Ham]
      
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== "http")
    
    if x̃
        push!(fauxPositifs, true)
    else
        push!(fauxPositifs, false)
    end
 
end

println("Sur les $(length(TestSet[:Ham])) hams de l'ensemble de test,")
println("- on en classe $(count(.!fauxPositifs)) comme ham")
println("- on en classe $(count(fauxPositifs)) comme spam.")


Sur les 1224 hams de l'ensemble de test,
- on en classe 1171 comme ham
- on en classe 53 comme spam.


### d) Filtrez tous les pourriels de l'échantillon de test. Quelle est la proportion de pourriels classés comme pourriels?

In [9]:
vraisPositifs = Bool[]

for filename in TestSet[:Spam]
   
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== "http")

    if x̃
        push!(vraisPositifs, true)
    else
        push!(vraisPositifs, false)
    end
    
end

println("Sur les $(length(TestSet[:Spam])) spams de l'ensemble de test,")
println("- on en classe $(count(.!vraisPositifs)) comme ham")
println("- on en classe $(count(vraisPositifs)) comme spam.")

Sur les 500 spams de l'ensemble de test,
- on en classe 356 comme ham
- on en classe 144 comme spam.


# Exercice 2 <a id = "unit2" > </a > 

Considérez les mots ***http*** et ***enron*** comme variables explicatives pour classer les messages électroniques en courriels et pourriels.

### a) Si un message contient les mots ***http*** et ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [10]:
# nombre de courriels où le mot enron apparaît
n₀₂ = ham_wordcounts["enron"]

# nombre de pourriels où le mot enron apparaît
if haskey(spam_wordcounts, "enron")
    n₁₂ = spam_wordcounts["enron"]
else
    n₁₂ = 0
end

q_spam = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2) * (n₁₂+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2) * (n₀₂+1)/(n₀+2)

p_spam = q_spam/(q_spam+q_ham)

println("Si les mots http et enron sont présents dans le message,")
println("il y a une prob de $p_spam que ce soit un spam.")

Si les mots http et enron sont présents dans le message,
il y a une prob de 0.009796468260453528 que ce soit un spam.


### b) Si un message ne contient pas les mots ***http*** et ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [11]:
q_spam = (n₁+1)/(n+2) * (n₁-n₁₁+1)/(n₁+2) * (n₁-n₁₂+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀-n₀₁+1)/(n₀+2) * (n₀-n₀₂+1)/(n₀+2)

p_spam = q_spam/(q_spam+q_ham)

println("Si les mots http et enron sont absents dans le message,")
println("il y a une prob de $p_spam que ce soit un spam.")

Si les mots http et enron sont absents dans le message,
il y a une prob de 0.3213865666050161 que ce soit un spam.


### c) Si un message contient le mot ***http*** mais ne contient pas le mot ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [12]:
q_spam = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2) * (n₁-n₁₂+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2) * (n₀-n₀₂+1)/(n₀+2)

p_spam = q_spam/(q_spam+q_ham)

println("Si le mot http est présent mais le mot enron est absent dans le message,")
println("il y a une prob de $p_spam que ce soit un spam.")

Si le mot http est présent mais le mot enron est absent dans le message,
il y a une prob de 0.8700033369884936 que ce soit un spam.


### d) Si un message ne contient pas le mot ***http*** mais contient le mot ***enron***, quelle est la probabilité que ce message soit un pourriel ?

In [13]:
q_spam = (n₁+1)/(n+2) * (n₁-n₁₁+1)/(n₁+2) * (n₁₂+1)/(n₁+2)
q_ham = (n₀+1)/(n+2) * (n₀-n₀₁+1)/(n₀+2) * (n₀₂+1)/(n₀+2)

p_spam = q_spam/(q_spam+q_ham)

println("Si le mot http est absent et le mot enron est présent dans le message,")
println("il y a une prob de $p_spam que ce soit un spam.")

Si le mot http est absent et le mot enron est présent dans le message,
il y a une prob de 0.0006996126828610966 que ce soit un spam.


### e) Filtrez tous les courriels de l'échantillon de test. Quelle est la proportion de courriels classés comme pourriels ?

In [14]:
fauxPositifs = Bool[]

for filename in TestSet[:Ham]
      
    wordlist = wordlisting(filename)
    x̃₁ = any(wordlist .== "http")
    x̃₂ = any(wordlist .== "enron")
    
    if x̃₁ && !x̃₂
        push!(fauxPositifs, true)
    else
        push!(fauxPositifs, false)
    end
 
end

println("Sur les $(length(TestSet[:Ham])) hams de l'ensemble de test,")
println("- on en classe $(count(.!fauxPositifs)) comme ham")
println("- on en classe $(count(fauxPositifs)) comme spam.")


Sur les 1224 hams de l'ensemble de test,
- on en classe 1195 comme ham
- on en classe 29 comme spam.


### f) Filtrez tous les pourriels de l'échantillon de test. Quelle est la proportion de pourriels classés comme pourriels ?

In [15]:
vraisPositifs = Bool[]

for filename in TestSet[:Spam]
      
    wordlist = wordlisting(filename)
    x̃₁ = any(wordlist .== "http")
    x̃₂ = any(wordlist .== "enron")
    
    if x̃₁ && !x̃₂
        push!(vraisPositifs, true)
    else
        push!(vraisPositifs, false)
    end
 
end

println("Sur les $(length(TestSet[:Spam])) spams de l'ensemble de test,")
println("- on en classe $(count(.!vraisPositifs)) comme ham")
println("- on en classe $(count(vraisPositifs)) comme spam.")


Sur les 500 spams de l'ensemble de test,
- on en classe 356 comme ham
- on en classe 144 comme spam.


# Exercice 3 <a id = "unit3" > </a > 

Pour cet exercice, nous utiliserons les mots les plus discriminant pour filtrer les messages. Pour identifier les mots les plus discriminants, l'information conjointe entre les mots et le classement est utilisée. L'information conjointe est une notion que nous verrons dans le dernier chapitre du cours concernant la théorie de l'information.


L'exécution de la première cellule de code de cette section vous donnera la variable ``discr_words``. Le premier mot de cette liste de mot correspond au mot le plus discriminant pour classer les messages en courriels et pourriels. Le deuxième mot de la liste est le second mot le plus discriminant et ainsi de suite.

In [16]:

struct BagOfWords
    n₀::Int               # Nombre de courriels de l'ensemble d"entraînement
    n₁::Int               # Nombre de pourriels de l'ensemble d"entraînement 
    ham_wordlist::Dict    # Occurrence des mots dans les hams
    spam_wordlist::Dict # Occurrence des mots dans les spams
end

function wordoccurrences(word::T,B::BagOfWords) where T<:AbstractString
    
    ham_wordcounts = B.ham_wordlist
    spam_wordcounts = B.spam_wordlist

    if haskey(ham_wordcounts, word)
        n10 = ham_wordcounts[word]
    else
        n10 = 0
    end

    if haskey(spam_wordcounts, word)
        n11 = spam_wordcounts[word]
    else
        n11 = 0
    end
    
    n = [n10, n11]
    
    return n
    
end

function mutualInformation(word::T,B::BagOfWords) where T<:AbstractString
   
    α = B.n₁ / (B.n₀ + B.n₁)
    
    n = wordoccurrences(word,B)
    
    θ₀₁ = (n[1]+1) / (B.n₀+2)
    θ₁₁ = (n[2]+1) / (B.n₁+2)
    
    θ₁ = (1-α)*θ₀₁ + α*θ₁₁ 
    
    I_mat = [ (1-α)*(1-θ₀₁)*log( (1-θ₀₁)/(1-θ₁) ), (1-α)*θ₀₁*log( θ₀₁/θ₁ ),
        α*(1-θ₁₁)*log( (1-θ₁₁)/(1-θ₁) ), α*θ₁₁*log( θ₁₁/θ₁ )  ]
    
    I = sum(I_mat)
    
    return I
    
end


wordBag = BagOfWords(length(TrainSet[:Ham]),length(TrainSet[:Spam]),ham_wordcounts,spam_wordcounts)

filenames = vcat(TrainSet[:Ham],TrainSet[:Spam])

wordlist = wordlisting.(filenames)

words = unique(vcat(wordlist...))

I = Float64[]

for word in words
   push!(I, mutualInformation(word,wordBag)) 
end

indperm = sortperm(I,rev=true)

discr_words = words[indperm]

36513-element Array{SubString{String},1}:
 "enron"     
 "cc"        
 "hpl"       
 "http"      
 "daren"     
 "gas"       
 "forwarded" 
 "pm"        
 "hou"       
 "ect"       
 "thanks"    
 "meter"     
 "subject"   
 ⋮           
 "timothy"   
 "darrin"    
 "activate"  
 "prizes"    
 "started"   
 "rest"      
 "having"    
 "okay"      
 "class"     
 "operations"
 "confirm"   
 "behalf"    

## a) En prenant les 10 mots les plus discriminant, quelle est la proportion de courriels de l'ensemble de test qui sont classés comme pourriels ?

In [17]:

function wordspamliness(word::T,occurs::Bool,B::BagOfWords) where T<:AbstractString
   
    n0 = B.n₀
    n1 = B.n₁
    n = n0 + n1
    
    ham_wordcounts = B.ham_wordlist
    spam_wordcounts = B.spam_wordlist

    if haskey(ham_wordcounts, word)
        n10 = ham_wordcounts[word]
    else
        n10 = 0
    end

    if haskey(spam_wordcounts, word)
        n11 = spam_wordcounts[word]
    else
        n11 = 0
    end
    
    q = Array{Float64}(undef,2)
    
    if occurs
        q[1] = log(n10+1)-log(n0+2)
        q[2] = log(n11+1)-log(n1+2)
    else
        q[1] = log(n0-n10+1)-log(n0+2)
        q[2] = log(n1-n11+1)-log(n1+2)
    end

    return q
    
end



function emailfiltering(filename::String,words::Array{T},B::BagOfWords) where T<:AbstractString
   
    n0 = B.n₀
    n1 = B.n₁
    n = n0 + n1
    
    wordlist = wordlisting(filename)
    
    p = [log(n0+1)-log(n+2), log(n1+1)-log(n+2)]
    
    for word in words
    
        x̃ = any(wordlist .== word)

        p = p + wordspamliness(word,x̃,B)
       
    end
    
    _, ind = findmax(p)
    
    if ind==1
        spam = false
    else
        spam = true
    end
    
    return spam
    
end



emailfiltering (generic function with 1 method)

In [18]:
filenames = TestSet[:Ham]
fauxPositifs = [emailfiltering(filenames[i],discr_words[1:10],wordBag) for i=1:length(filenames)]
println("Sur les $(length(filenames)) hams de l'ensemble de test,")
println("- on en classe $(count(.!fauxPositifs)) comme ham")
println("- on en classe $(count(fauxPositifs)) comme spam.")

Sur les 1224 hams de l'ensemble de test,
- on en classe 925 comme ham
- on en classe 299 comme spam.


## b) En prenant les 10 mots les plus discriminant, quelle est la proportion de pourriels de l'ensemble de test qui sont classés comme pourriels ?

In [19]:
filenames = TestSet[:Spam]
vraisPositifs = [emailfiltering(filenames[i],discr_words[1:10],wordBag) for i=1:length(filenames)]
println("Sur les $(length(filenames)) spams de l'ensemble de test,")
println("- on en classe $(count(.!vraisPositifs)) comme ham")
println("- on en classe $(count(vraisPositifs)) comme spam.")

Sur les 500 spams de l'ensemble de test,
- on en classe 16 comme ham
- on en classe 484 comme spam.


## c) En prenant les 100 mots les plus discriminant, quelle est la proportion de courriels de l'ensemble de test qui sont classés comme pourriels ?

In [20]:
filenames = TestSet[:Ham]
fauxPositifs = [emailfiltering(filenames[i],discr_words[1:100],wordBag) for i=1:length(filenames)]
println("Sur les $(length(filenames)) hams de l'ensemble de test,")
println("- on en classe $(count(.!fauxPositifs)) comme ham")
println("- on en classe $(count(fauxPositifs)) comme spam.")

Sur les 1224 hams de l'ensemble de test,
- on en classe 1078 comme ham
- on en classe 146 comme spam.


## d) En prenant les 100 mots les plus discriminant, quelle est la proportion de pourriels de l'ensemble de test qui sont classés comme pourriels ?

In [21]:
filenames = TestSet[:Spam]
vraisPositifs = [emailfiltering(filenames[i],discr_words[1:100],wordBag) for i=1:length(filenames)]
println("Sur les $(length(filenames)) spams de l'ensemble de test,")
println("- on en classe $(count(.!vraisPositifs)) comme ham")
println("- on en classe $(count(vraisPositifs)) comme spam.")

Sur les 500 spams de l'ensemble de test,
- on en classe 3 comme ham
- on en classe 497 comme spam.


## e) Quel serait le nombre idéal de mots discriminant qu'il faudrait prendre ?

Il n'y a pas une seule réponse possible. Ça dépend si on veut limiter le nombre de faux positifs ou le nombre de faux négatifs.